<a href="https://colab.research.google.com/github/szhang06/energy_price_prediction/blob/main/Energy_Price_Prediction_Sagemaker_XGB_version_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Build an Electricity Price Forecasting System with Amazon Sagemaker Xgboost**

##### Note: the data processing is handled in the other notebook for cost saving. The processed dataset is included in the file.


## Import liabraries


In [ ]:
!pip install sagemaker -U

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 654.9/654.9 KB 11.4 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 48.2 MB/s eta 0:00:0000:0100:01
  Created wheel for sagemaker: filename=sagemaker-2.126.0-py2.py3-none-any.whl size=890086 sha256=ef51764e62a6a4ad4e496d805bd318eaa054939b5a976be648b85e7a13c36e49
  Stored in directory: /home/ec2-user/.cache/pip/wheels/2f/4e/63/345e2f96c60d3f77a2b8be1182a430341092f763b4479dc578
Successfully built sagemaker
  Attempting uninstall: botocore
    Found existing installation: botocore 1.24.19
    Uninstalling botocore-1.24.19:
      Successfully uninstalled botocore-1.24.19
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.125.0
    Uninstalling sagemaker-2.125.0:
      Successfully uninstalled sagemaker-2.125.0
ERROR: pip's dependency resolver does not currently ta

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error
import sagemaker
import boto3
# from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session

In [ ]:
role = sagemaker.get_execution_role()

In [ ]:
print(role)

arn:aws:iam::751247534603:role/service-role/SageMaker-DataScientist


In [ ]:
bucket_name = sagemaker.Session().default_bucket()
print(bucket_name)

sagemaker-us-east-1-751247534603


In [ ]:
# bucket_name = "energy-bucket"
my_region = boto3.session.Session().region_name # my region
print(my_region)

us-east-1


In [ ]:
# path to save trained model
prefix = 'xgboost-energy'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://sagemaker-us-east-1-751247534603/xgboost-energy/output


## Obtain, split data and save it to the bucket

In [ ]:
import pandas as pd

In [ ]:
train_data, validate_data, test_data = [pd.read_csv("./" + name) for name in ["train.csv", "dev.csv", "test.csv"]]
# print(train_data.shape, validate_data.shape, test_data.shape)

In [ ]:
train_data.head()

,64.92,449.0,328.0,5196.0,4755.0,158.0,920.0,1009.0,1658.0,7096.0,...,71.0,50.0,195.0,5890.0,24382.0,48.1,0,1,1.1,2015
0,64.48,448.0,323.0,4857.0,4581.0,157.0,1164.0,973.0,1371.0,7099.0,...,73.0,50.0,196.0,5461.0,22734.0,47.33,1,1,1,2015
1,59.32,438.0,254.0,4314.0,4131.0,160.0,1503.0,949.0,779.0,7098.0,...,75.0,50.0,191.0,5238.0,21286.0,42.27,2,1,1,2015
2,56.04,428.0,187.0,4130.0,3840.0,156.0,1826.0,953.0,720.0,7097.0,...,74.0,42.0,189.0,4935.0,20264.0,38.41,3,1,1,2015
3,53.63,410.0,178.0,4038.0,3590.0,156.0,2109.0,952.0,743.0,7098.0,...,74.0,34.0,188.0,4618.0,19905.0,35.72,4,1,1,2015
4,51.73,401.0,172.0,4040.0,3368.0,158.0,2108.0,961.0,848.0,7098.0,...,74.0,34.0,186.0,4397.0,20010.0,35.13,5,1,1,2015


In [ ]:
validate_data.head()

,57.1,376.0,584.0,6728.0,6752.0,298.0,0.0,1016.0,1854.0,5052.0,...,103.0,4234.0,299.0,3773.0,32013.0,50.34,13,26,5,2017
0,56.64,374.0,585.0,6921.0,6758.0,296.0,0.0,1021.0,1602.0,5048.0,...,101.0,3815.0,302.0,4130.0,31924.0,50.15,14,26,5,2017
1,56.53,373.0,586.0,7087.0,6752.0,294.0,0.0,1023.0,1409.0,5051.0,...,109.0,3182.0,313.0,4491.0,31912.0,50.15,15,26,5,2017
2,56.61,384.0,585.0,7121.0,6717.0,295.0,0.0,1025.0,1044.0,5050.0,...,111.0,2488.0,320.0,5108.0,31447.0,50.15,16,26,5,2017
3,57.31,395.0,585.0,7065.0,6698.0,296.0,0.0,1029.0,1128.0,5050.0,...,111.0,1761.0,318.0,4817.0,30943.0,50.54,17,26,5,2017
4,58.18,396.0,585.0,6853.0,6721.0,311.0,0.0,1036.0,1761.0,5052.0,...,111.0,892.0,321.0,4905.0,30613.0,51.47,18,26,5,2017


In [ ]:
test_data.head()

,50.95,251.0,0.0,5319.0,1068.0,345.0,761.0,1849.0,6391.0,6068.0,...,102.0,31.0,277.0,14265.0,35044.0,37.61,18,14,3,2018
0,59.73,252.0,0.0,5292.0,1126.0,351.0,149.0,1821.0,6756.0,6068.0,...,101.0,27.0,290.0,13306.0,36230.0,48.21,19,14,3,2018
1,50.89,256.0,0.0,4318.0,1054.0,351.0,168.0,1853.0,6104.0,6068.0,...,103.0,17.0,294.0,12970.0,35065.0,41.11,20,14,3,2018
2,45.94,254.0,0.0,4047.0,963.0,350.0,811.0,1853.0,5347.0,6073.0,...,104.0,17.0,288.0,13090.0,32328.0,36.72,21,14,3,2018
3,30.59,248.0,0.0,3726.0,853.0,329.0,2007.0,1826.0,5237.0,6074.0,...,95.0,17.0,288.0,13408.0,29528.0,21.20,22,14,3,2018
4,32.54,236.0,0.0,3359.0,783.0,283.0,3283.0,1812.0,4312.0,6074.0,...,91.0,17.0,278.0,13406.0,27513.0,25.36,23,14,3,2018


In [ ]:
# save train data into bucket

import os

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')
     

In [ ]:
# save validate data into bucket

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'dev/dev.csv')).upload_file('dev.csv')
s3_input_dev = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/dev'.format(bucket_name, prefix), content_type='csv')
     

In [ ]:
# save test data into bucket

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')
     

# Build and Train xgboost regression - built-in

In [ ]:
# get xgb container
# repo_version based on preference
# container = get_image_uri(boto3.Session().region_name,
#                           'xgboost', 
#                           repo_version='1.0-1')

container = sagemaker.image_uris.retrieve("xgboost", boto3.Session().region_name, "latest") # version matters as it causes missing hyperparameter errors, etc.

In [ ]:

# # hyperparameters already tuned locally to save billing cost

# hyperparameters = {
#         'colsample_bytree': "0.5", 
#         'learning_rate': "0.3",
#         'max_depth': "6",
#         'min_child_weight': "5",
# #         'n_estimators': "1000",
#         'objective': 'reg:squarederror',
#         'subsample': 0.5,
#         "num_round":1000
#     }

In [ ]:
# call xgb container
estimator = sagemaker.estimator.Estimator(container, 
#                                           hyperparameters=hyperparameters,
                                          role,
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          
#                                           instance_type=' ml.t3.medium',
                                          volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          
                                          use_spot_instances=True,
                                          max_run=300,
                                          max_wait=600) # last three arguments for saving billing cost

estimator.set_hyperparameters(
    eval_metric="rmse",
    objective="reg:linear",
    num_round=1500, # no of estimators, tunable below? ->  missing
    rate_drop=0.3,
    tweedie_variance_power=1.4,
    
    
)

In [ ]:
# hyperparameter tuning

# https://catalog.us-east-1.prod.workshops.aws/workshops/63069e26-921c-4ce1-9cc7-dd882ff62575/en-US/lab2#automatic-model-tuning-(optional)


from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {
    'eta': ContinuousParameter(0.05, 0.3), # learning rate
    'min_child_weight': ContinuousParameter(1, 10),
    'alpha': ContinuousParameter(0, 2), # regularization
    'max_depth': IntegerParameter(5, 8),
    'subsample': ContinuousParameter(0.5, 0.9)
    
}

objective_metric_name = "validation: rsme" 

In [ ]:
# create tuning object

tuner = HyperparameterTuner(
    estimator, objective_metric_name, hyperparameter_ranges, max_jobs=20, max_parallel_jobs=3, early_stopping_type="Auto"
)

In [ ]:
# launch tuning
"""
y is the first column


"""
estimator.fit({'train': s3_input_train,'validation': s3_input_dev})

INFO:sagemaker:Creating training-job with name: xgboost-2022-12-27-16-29-02-634


2022-12-27 16:29:02 Starting - Starting the training job...
2022-12-27 16:29:17 Starting - Preparing the instances for training.........
2022-12-27 16:30:53 Downloading - Downloading input data..Arguments: train
[2022-12-27:16:31:16:INFO] Running standalone xgboost training.
[2022-12-27:16:31:16:INFO] File size need to be processed in the node: 3.16mb. Available memory size in the node: 24026.41mb
[2022-12-27:16:31:16:INFO] Determined delimiter of CSV input is ','
[16:31:16] S3DistributionType set as FullyReplicated
[16:31:16] 21037x20 matrix with 420740 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2022-12-27:16:31:16:INFO] Determined delimiter of CSV input is ','
[16:31:16] S3DistributionType set as FullyReplicated
[16:31:16] 7013x20 matrix with 140260 entries loaded from /opt/ml/input/data/validation?format=csv&label_column=0&delimiter=,
[16:31:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 108 extra nodes, 0 pruned nodes, max_dept


2022-12-27 16:31:14 Training - Training image download completed. Training in progress.[16:31:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[360]#011train-rmse:0.846635#011validation-rmse:8.23986
[16:31:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 98 extra nodes, 0 pruned nodes, max_depth=6
[361]#011train-rmse:0.844544#011validation-rmse:8.24284
[16:31:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=6
[362]#011train-rmse:0.840124#011validation-rmse:8.24422
[16:31:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 98 extra nodes, 0 pruned nodes, max_depth=6
[363]#011train-rmse:0.838353#011validation-rmse:8.24417
[16:31:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 104 extra nodes, 0 pruned nodes, max_depth=6
[364]#011train-rmse:0.834478#011validation-rmse:8.24375
[16:31:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1

[969]#011train-rmse:0.227205#011validation-rmse:8.25498
[16:31:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 80 extra nodes, 0 pruned nodes, max_depth=6
[970]#011train-rmse:0.227033#011validation-rmse:8.25501
[16:31:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 92 extra nodes, 0 pruned nodes, max_depth=6
[971]#011train-rmse:0.226925#011validation-rmse:8.255
[16:31:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 88 extra nodes, 0 pruned nodes, max_depth=6
[972]#011train-rmse:0.226601#011validation-rmse:8.25501
[16:31:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 0 pruned nodes, max_depth=6
[973]#011train-rmse:0.226262#011validation-rmse:8.25506
[16:31:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 104 extra nodes, 0 pruned nodes, max_depth=6
[974]#011train-rmse:0.225745#011validation-rmse:8.25514
[16:31:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 102 extra nodes, 0 pruned nodes, max_


2022-12-27 16:31:45 Uploading - Uploading generated training model
2022-12-27 16:31:45 Completed - Training job completed
Training seconds: 52
Billable seconds: 19
Managed Spot Training savings: 63.5%


# Deploy the model as Endpoints

In [ ]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: xgboost-2022-12-27-16-32-17-753
INFO:sagemaker:Creating endpoint-config with name xgboost-2022-12-27-16-32-17-753
INFO:sagemaker:Creating endpoint with name xgboost-2022-12-27-16-32-17-753


--------!

# Prediction of Test Data

In [ ]:
from sagemaker.predictor import csv_serializer
test_data_array = test_data.iloc[:,1:].values #load the data into an array
# xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = csv_serializer # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


(7012,)


In [ ]:
print("predictions: ", predictions_array)

predictions:  [ 0.93906021 48.7353363  46.89014816 ... 71.50193024 74.45533752
 75.73748016]


In [ ]:
print(np.sqrt(mean_squared_error(test_data.iloc[:,0].values, predictions_array)))

7.688643995283099


# Delete Endpoint

In [ ]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Deleting endpoint with name: xgboost-2022-12-27-16-32-17-753
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


[{'ResponseMetadata': {'RequestId': 'SJKFW7F0R649CPX4',
   'HostId': 'yVFKyzDAi7W6VRgDdTy8nAlm1dLyO1l6qXQIWlxe14w693ZBaljDRbpFku593tSiqQ7sNPkD3zU=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'yVFKyzDAi7W6VRgDdTy8nAlm1dLyO1l6qXQIWlxe14w693ZBaljDRbpFku593tSiqQ7sNPkD3zU=',
    'x-amz-request-id': 'SJKFW7F0R649CPX4',
    'date': 'Tue, 27 Dec 2022 16:53:46 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-energy/output/xgboost-2022-12-27-00-15-24-669/debug-output/training_job_end.ts'},
   {'Key': 'xgboost-energy/output/xgboost-2022-12-27-16-29-02-634/debug-output/training_job_end.ts'},
   {'Key': 'xgboost-energy/train/train.csv'},
   {'Key': 'xgboost-energy/output/xgboost-2022-12-27-00-07-56-890/debug-output/training_job_end.ts'},
   {'Key': 'xgboost-energy/output/xgboost-2022-12-27-01-46-48-867/output/model.tar.gz'},
   {'Key': 'xgboos